## Step 0: Import requirements

In [1]:
from config import GOOGLE_PLACES_API_KEY # constants are uppercase
import requests 
import pandas as pd
from pandas.io.json import json_normalize
import gmplot
from ast import literal_eval
from palettable.colorbrewer.diverging import Spectral_9 as color_scheme
import os

## Step 1: Get places location from keywords

In [2]:
SF_LOCATION = '37.773972,-122.431297'
KEYWORD='Whole Foods Market'

In [3]:
def locate_places(city_location, keyword):
    """
    Using the Google Places API and the requests library, search for a specific keyword in a specified 
    location (the radius of a point), and return the latitude and longitude of the top 20 results (ranked by prominence)
    in a dataframe
    """

    # defining the api-endpoint  
    GOOGLE_PLACES_API_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # data to be sent to api 
    data = {'location': city_location, # location — The latitude/longitude around which to retrieve place information. This must be specified as latitude,longitude.
            'radius':50000, # radius — Defines the distance (in meters) within which to return place results. The maximum allowed radius is 50 000 meters.
            'keyword': keyword, # keyword — A term to be matched against all content that Google has indexed for this place, including but not limited to name, type, and address, as well as customer reviews and other third-party content.
            'key':GOOGLE_PLACES_API_KEY # key parameter must be last in the dictionary
           }

    # sending post request and saving response as response object 
    r = requests.get(url = GOOGLE_PLACES_API_URL, params = data) 

    # extracting response json  
    results_json = r.json()
    
    results_df = json_normalize(results_json['results'])

    return results_df

In [4]:
places_df = locate_places(SF_LOCATION, KEYWORD)
places_df.head()

,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,opening_hours.open_now,...,place_id,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity
0,37.380843,-121.976194,37.382383,-121.974827,37.379684,-121.977526,https://maps.gstatic.com/mapfiles/place_api/ic...,4c197b045d638674b239399ce2c96ed7dd385690,Whole Foods Market,True,...,ChIJ6SikCPLJj4ARydxZnnuq98M,"92JF+8G Santa Clara, California",849W92JF+8G,3,4.4,ChIJ6SikCPLJj4ARydxZnnuq98M,GOOGLE,"[grocery_or_supermarket, supermarket, food, he...",1875,"2732 Augustine Dr Ste 1600, Santa Clara"
1,37.768353,-122.427127,37.769850,-122.425621,37.767150,-122.428321,https://maps.gstatic.com/mapfiles/place_api/ic...,10c0ee72372bc6f02eafc51047a22f882b409200,Whole Foods Market,True,...,ChIJMdstMx5-j4ARTiRtCtd6LFg,"QH9F+84 Mission District, San Francisco, CA",849VQH9F+84,3,4.2,ChIJMdstMx5-j4ARTiRtCtd6LFg,GOOGLE,"[grocery_or_supermarket, supermarket, food, he...",845,"2001 Market St, San Francisco"
2,37.398934,-122.110648,37.400472,-122.109296,37.397773,-122.111996,https://maps.gstatic.com/mapfiles/place_api/ic...,5deed91ed35ceb88c1635531b7137a226aaf9814,Whole Foods Market,True,...,ChIJwz4tvpmwj4ARvlDtWYF38h4,"9VXQ+HP Los Altos, California",849V9VXQ+HP,3,4.4,ChIJwz4tvpmwj4ARvlDtWYF38h4,GOOGLE,"[grocery_or_supermarket, supermarket, food, he...",1389,"4800 El Camino Real, Los Altos"
3,37.789951,-122.423101,37.791457,-122.422011,37.788757,-122.424711,https://maps.gstatic.com/mapfiles/place_api/ic...,1cbdc9e90161a92283b0637131153462a748ea63,Whole Foods Market,True,...,ChIJC55dsuqAhYARQApjF1OPPWU,"QHQG+XQ Western Addition, San Francisco, CA",849VQHQG+XQ,3,4.1,ChIJC55dsuqAhYARQApjF1OPPWU,GOOGLE,"[grocery_or_supermarket, supermarket, food, he...",1062,"1765 California St, San Francisco"
4,37.332170,-121.904980,37.333555,-121.903206,37.330855,-121.905906,https://maps.gstatic.com/mapfiles/place_api/ic...,f18bf04bf8a2855e9da20b5661dea8696c8d6f92,Whole Foods Market,True,...,ChIJOzlY-l7Lj4ARIQXu7CsRNmY,"83JW+V2 San Jose, California",849W83JW+V2,3,4.5,ChIJOzlY-l7Lj4ARIQXu7CsRNmY,GOOGLE,"[grocery_or_supermarket, supermarket, food, he...",1387,"777 The Alameda, San Jose"


## Step 2: Plot points of interest onto a map

In [5]:
city_lat, city_lng = literal_eval(SF_LOCATION)

In [6]:
locations = places_df[['geometry.location.lat', 'geometry.location.lng']].to_numpy()
locations

array([[  37.3808426, -121.9761941],
       [  37.7683532, -122.4271267],
       [  37.3989341, -122.1106483],
       [  37.7899512, -122.4231006],
       [  37.3321699, -121.90498  ],
       [  37.4424053, -122.1596147],
       [  37.7812561, -122.3998965],
       [  37.554994 , -121.9852099],
       [  37.5441581, -122.2920129],
       [  37.7057284, -121.8897911],
       [  37.7237983, -122.4545528],
       [  37.7645658, -122.4029773],
       [  37.8127708, -122.2609344],
       [  37.7617624, -121.9612642],
       [  37.7519697, -122.4308871],
       [  37.4822223, -122.2316198],
       [  37.9741361, -122.0588067],
       [  37.9713437, -122.5168364],
       [  37.769638 , -122.4528278],
       [  37.855655 , -122.2604434]])

In [7]:
layer_counter = 0
color = color_scheme.hex_colors[layer_counter]

In [8]:
def plot_places(city_location, keywords_list, map_html_file):

    city_lat, city_lng = literal_eval(city_location)
    
    gmap = gmplot.GoogleMapPlotter(city_lat, city_lng, 13) # latitude, longitude, zoom level
   
    layer_counter = 0
    for keyword in keywords_list:
            
        places_df = locate_places(city_location, keyword)
    
        points_lat = places_df['geometry.location.lat']
        points_lng = places_df['geometry.location.lng']
        
        color = color_scheme.hex_colors[layer_counter]
        gmap.scatter(points_lat, points_lng, color, size = 60, marker=False)
        
        layer_counter += 1
        
    gmap.apikey = GOOGLE_PLACES_API_KEY
    gmap.draw(map_html_file)
    
    os.system("start " + map_html_file)

In [9]:
map_html_file = '../reports/figures/gmplot.html'
plot_places(SF_LOCATION, ['Grocery', 'Resaurant', 'School'], map_html_file)

In [10]:
def plot_places(city_location, map_html_file):
    city_lat, city_lng = literal_eval(city_location)    
    gmap = gmplot.GoogleMapPlotter(city_lat, city_lng, 13) # latitude, longitude, zoom level
    gmap.apikey = GOOGLE_PLACES_API_KEY


    i = 0
    while True:
        keyword = input("Enter a keyword to search for in this area: ")
        if keyword == 'lemme outta here':
            break
        places_df = locate_places(city_location, keyword)
        points_lat = places_df['geometry.location.lat']
        points_lng = places_df['geometry.location.lng']
        color = color_scheme.hex_colors[i]
        gmap.scatter(points_lat, points_lng, color, size = 60, marker=False)
        i += 1
        gmap.draw(map_html_file)
        os.system("start " + map_html_file)

In [11]:
plot_places(SF_LOCATION, map_html_file)

Enter a keyword to search for in this area: lemme outta here
